# Deploying the model

<p>&nbsp;</p>
<p><span style="font-weight: 400;">Our data science team is very happy with the results. They were able to build a dataset, create features, train multiple models, conduct hyper parameter optimization, receive an AUC of almost 1.0 (!), evaluate the model and make sure it&rsquo;s not overfitting. The management thinks that they must have worked so hard to achieve it, but in fact, it was very easy to achieve with AWS&rsquo;s ML and Data ecosystem :)</span></p>
<p>&nbsp;</p>
<p><span style="font-weight: 400;">Now, before collecting their project bonus they just need to make sure they deliver valuable products to the company&rsquo;s business. In order to do that they would be required to achieve the following:</span></p>
<ul>
<li><span style="color: #333399;"><strong>Deploy the model to production</strong></span><span style="font-weight: 400;"> -&nbsp; make sure the deployment leverages the insights from the analysis, train the model on the entire data available (Train+Test) while decreasing the amount of regularization used.</span></li>
<li><span style="color: #333399;"><strong>Track versions of the models in production</strong></span></li>
<li><span style="color: #333399;"><strong>Monitor predictions on incoming data </strong></span><span style="font-weight: 400;">- keep track of the new samples that come in. Identify when the accuracy drops and analyze the cause.</span></li>
<li><strong><span style="color: #333399;">Secure model artifacts</span> </strong>- <span style="font-weight: 400;">protect the company&rsquo;s IP by making sure that only authorized calls can invoke the model and protect the data.</span></li>
<li><span style="color: #333399;"><strong>Stabilize the service</strong></span> - increase the reliability of the serving layer in order to serve incoming requests with high reliability and low latency. Avoid local service disruptions implications on the application and scale up and down according to the influx of requests.</li>
</ul>

In [1]:
import yaml
from sqlalchemy import create_engine, text
import pandas as pd
from datetime import datetime
import boto3
import re
import os
import json
from sagemaker.estimator import Estimator
from sagemaker import get_execution_role, session, Session
from sagemaker.model_monitor import ModelQualityMonitor
from sagemaker.inputs import TrainingInput
from sagemaker import image_uris
role = get_execution_role()
session = Session()
region = boto3.Session().region_name


## Train the model for production 

<p><span style="font-weight: 400;">When training the model to production, the team utilizes the optimal parameters found using AWS SageMaker&rsquo;s Hyperparameter Optimization service. They now also include in the training set the data from the original train and test data. A few points they keep in mind:</span></p>
<ul>
<li style="font-weight: 400;" aria-level="1"><span style="font-weight: 400;">The model is trained under a<strong> different AWS account</strong>, using different resources used for the research phase (bucket, artifacts etc).</span></li>
<li style="font-weight: 400;" aria-level="1"><span style="font-weight: 400;">The training data is now increased and may require distributed training. The training set will be split into smaller parts in order to allow <span style="color: #ff6600;"><strong>SageMaker&rsquo;s distributed</strong></span> version of <strong>XGBoost</strong> to be trained efficiently. </span></li>
</ul>

### Load the entire dataset and split it to shards 

In [2]:
conf = yaml.load(open('./vars.yaml', 'r'))
rs_username = conf['REDSHIFT_USER']
rs_password = conf['REDSHIFT_PASSWORD']
rs_db = conf['REDSHIFT_DB']
rs_host = conf['REDSHIFT_HOST']
bucket = conf['PROD_BUCKET']

In [3]:
engine = create_engine(f'redshift+psycopg2://{rs_username}:{rs_password}@{rs_host}:5439/{rs_db}')

In [4]:
sql_get_dataset = '''
SELECT
isfraud
, amount
, diff_dest_equal_amount
, diff_origin_equal_amount
, is_cash_in
, is_cash_out
, is_debit
, is_payment
, is_transfer
, namedest_c
, nameorig_c
, newbalancedest
, newbalanceorig
, oldbalancedest
, oldbalanceorg
, step
FROM dataset
WHERE step <= 399
'''

In [5]:
df = pd.read_sql(sql_get_dataset, con=engine)

In [6]:
idxs = pd.Series(df.index).quantile([0,0.2,0.4,0.6,0.8,1]).astype(int).tolist()

In [7]:
train_path = f's3://{bucket}/train/'

In [8]:
def save_df(df_to_save, part):
    save_path = os.path.join(train_path, f'train_{part}.csv')
    df_to_save.drop('step', axis=1).to_csv(save_path , index=False, header=None)
    
for part in range(len(idxs) - 1):
    save_df(df.iloc[idxs[part]:idxs[part+1]], part)

### Submit a distributed training job

In [9]:
s3_input_train = TrainingInput(s3_data=train_path, content_type='csv',distribution='ShardedByS3Key')

In [10]:
container = image_uris.retrieve('xgboost', boto3.Session().region_name, 'latest')

In [11]:
xgb = Estimator(container, role,  instance_count=5,  instance_type='ml.m4.xlarge', 
                output_path=f's3://{bucket}/models/output', sagemaker_session=session)

In [12]:
xgb.set_hyperparameters(eval_metric='auc',
                        objective='binary:logistic',
                        num_round=1000,
                        rate_drop=0.3,
                        eta=0.7686093376581404,
                        min_child_weight=29.85,
                        alpha=0.6698792306761945,
                        max_depth=13)

In [13]:
training_job = xgb.fit(inputs={'train': s3_input_train, 'validation': s3_input_train}, wait=False)

## Managing models in production

In [14]:
sm_client = boto3.client('sagemaker', region_name=region)

### Group the model packages 

In [34]:
model_package_group_name = "fraud-detector-model-group-capstone"
model_package_group_input_dict = {
 "ModelPackageGroupName" : model_package_group_name,
 "ModelPackageGroupDescription" : "Fraud detection algorithhm model group"
}

create_model_pacakge_group_response = sm_client.create_model_package_group(**model_package_group_input_dict)
print('ModelPackageGroup Arn : {}'.format(create_model_pacakge_group_response['ModelPackageGroupArn']))

ModelPackageGroup Arn : arn:aws:sagemaker:us-east-1:404163662382:model-package-group/fraud-detector-model-group-capstone


In [35]:
package_name = 'fraud-detector-model-package'

In [36]:
location_of_inference_image = '811284229777.dkr.ecr.us-east-1.amazonaws.com/xgboost:latest'
location_of_model_artifacts = f's3://{bucket}/models/output/xgboost-2021-01-07-22-53-51-919/output/model.tar.gz'

In [37]:
modelpackage_inference_specification =  { 
    "InferenceSpecification": {    
      "Containers": [ { "Image": location_of_inference_image,
                      "ModelDataUrl": location_of_model_artifacts}],
      "SupportedContentTypes": [ "text/csv" ],
      "SupportedResponseMIMETypes": [ "text/csv" ],
   }
 }

In [38]:
create_model_package_input_dict = {
    "ModelPackageGroupName" : model_package_group_name,
    "ModelApprovalStatus" : "PendingManualApproval"
}
create_model_package_input_dict.update(modelpackage_inference_specification)

In [39]:
create_mode_package_response = sm_client.create_model_package(**create_model_package_input_dict)
model_package_arn = create_mode_package_response["ModelPackageArn"]
print('ModelPackage Version ARN : {}'.format(model_package_arn))

ModelPackage Version ARN : arn:aws:sagemaker:us-east-1:404163662382:model-package/fraud-detector-model-group-capstone/1


In [40]:
# Approve model package
model_package_update_input_dict = {
    "ModelPackageArn" : model_package_arn,
    "ModelApprovalStatus" : "Approved"
}
model_package_update_response = sm_client.update_model_package(**model_package_update_input_dict)

In [41]:
model_name = 'fraud-detection-model-' + datetime.utcnow().strftime("%Y-%m-%d-%H-%M-%S")
print("Model name : {}".format(model_name))
primary_container = {
    'ModelPackageName': model_package_arn,
}
create_model_respose = sm_client.create_model(
    ModelName = model_name,
    ExecutionRoleArn = role,
    PrimaryContainer = primary_container
)
print("Model arn : {}".format(create_model_respose["ModelArn"]))

Model name : fraud-detection-model-2021-01-10-16-39-34
Model arn : arn:aws:sagemaker:us-east-1:404163662382:model/fraud-detection-model-2021-01-10-16-39-34


### Endpoint configuration 

In [42]:
from sagemaker.model_monitor import DataCaptureConfig

In [43]:
data_capture_uri = f's3://{bucket}/data/samples_capture'

In [44]:
endpoint_config_name = 'fraud-detection-EndpointConfig-' + datetime.utcnow().strftime("%Y-%m-%d-%H-%M-%S")
print(endpoint_config_name)

fraud-detection-EndpointConfig-2021-01-10-16-39-39


In [45]:
prod_vars = [{'VariantName': 'AllTraffic',
              'ModelName': model_name,
              'InitialInstanceCount': 1,
              'InstanceType':'ml.m4.xlarge',
              'InitialVariantWeight':1,}]

In [52]:
data_capture_configuration = {
    "EnableCapture": True,
    "InitialSamplingPercentage": 100,
    "DestinationS3Uri": data_capture_uri,
    "CaptureOptions": [
        { "CaptureMode": "Output" },
        { "CaptureMode": "Input" }
    ],
    "CaptureContentTypeHeader": {
       "CsvContentTypes": ["text/csv"],
       "JsonContentTypes": ["application/json"]
}}

In [53]:
endpoint_conf = sm_client.create_endpoint_config(
      EndpointConfigName=endpoint_config_name,
      ProductionVariants=prod_vars ,
      DataCaptureConfig=data_capture_configuration)

In [54]:
endpoint_name = 'fraud-detection-endpoint-' + datetime.utcnow().strftime("%Y-%m-%d-%H-%M-%S")
print("EndpointName={}".format(endpoint_name))

create_endpoint_response = sm_client.create_endpoint(
    EndpointName=endpoint_name,
    EndpointConfigName=endpoint_config_name)
print(create_endpoint_response['EndpointArn'])

EndpointName=fraud-detection-endpoint-2021-01-10-16-42-59
arn:aws:sagemaker:us-east-1:404163662382:endpoint/fraud-detection-endpoint-2021-01-10-16-42-59


## Use the endpoint!

In [55]:
import io
from io import StringIO

In [56]:
runtime = boto3.Session().client('sagemaker-runtime')

In [57]:
new_data_path = f"s3://{conf['DATA_BUCKET']}/data/new_data/new_data.csv"
df_new_data = pd.read_csv(new_data_path, header=None)
df_new_data.columns = ['isfraud', 'amount', 'diff_dest_equal_amount', 'diff_origin_equal_amount',
                       'is_cash_in', 'is_cash_out', 'is_debit', 'is_payment',
                       'is_transfer', 'namedest_c', 'nameorig_c', 'newbalancedest', 
                       'newbalanceorig', 'oldbalancedest', 'oldbalanceorg']

In [58]:
csv_file = io.StringIO()
df_new_data.drop('isfraud', axis=1).sample(15).to_csv(csv_file, header=False, index=False)
payload_data = csv_file.getvalue()

In [61]:
response = runtime.invoke_endpoint(EndpointName=endpoint_name, ContentType='text/csv', Body=payload_data)
csv_file.close()

In [62]:
results = response['Body'].read().decode().split(',')
results = [float (x) for x in results]

In [63]:
results

[3.30434249918e-06,
 9.0295515065e-07,
 3.90222612623e-06,
 4.32596752944e-05,
 3.90222612623e-06,
 3.90222612623e-06,
 7.93489743955e-06,
 1.59437888669e-06,
 3.90222612623e-06,
 1.38526747833e-05,
 3.90222612623e-06,
 3.06003948936e-06,
 7.14734414942e-06,
 1.2759420315e-05,
 9.10034486878e-06]